# Using a TurtleBot3 Waffle with ROS

## Getting started

### Notes
**1:** *When using ROS you will need to have several Linux Terminal Windows running simultaneously.  Only close down terminals or shut down processes (using `CTRL+C`) when instructed to do so otherwise your robot may not run!*  
**2:** *Any text `in a grey box` below is a command that you should enter in the terminal to launch a ROS process, enter all of the text*  
**3:** *You can copy the text from here using `CTRL+C`, but to paste it into the terminal you will need to use `CTRL+SHIFT+V`*


1. First, make sure that the robot is turned on via the switch on the front
1. Make sure the laptop is connected to the *DIA-LAB* WiFi network  
1. Open a terminal window on the laptop by using the `CTRL+ALT+T` keyboard shortcut.  We'll refer to this as **[TERMINAL 1]**.  
1. Enter the following to establish a connection to your robot via ssh:  
**[TERMINAL 1]:** `ssh waffle@dia-waffleX`  
where `X` is the number of the robot as printed on the top.  Enter the password when requested (we'll provide this separately)
1. The terminal prompt should then change as follows:  
        waffle@dia-waffleX:~$  
which indicates that we are now logged in to a *"remote terminal"* running on the robot computer.  Text to the right of the `@` tells us the name of the remote computer (`dia-waffleX`), and text to the left of `@` tells us the name of the user that we are logged in as (`waffle` in this case)
1. Then launch `tmux` to make the remote terminal a bit easier to work with:  
**[TERMINAL 1]:** `tmux`  
Any text in the terminal should now clear and a green banner should appear across the bottom of the window indicating that a *tmux* session is active.  We'll call this terminal window **[TERMINAL 1][TMUX 0]** from here on in 
1. To get ROS up and running we need to launch the *ROS master* in a new terminal **on the laptop**.  Open up a **new terminal window** on the laptop using `CTRL+ALT+T` (which we'll call **[TERMINAL 2]**)
1. Use `roscore` to launch the ROS master:  
**[TERMINAL 2]:** `roscore`  
A terminal message ending as follows:  
        started core services [/rosout]  
indicates that the ROS Master has been launched successfully  
1. Then we need to launch all the core *ROS packages* on the robot.  Return to **[TERMINAL 1][TMUX 0]** and enter the following:  
**[TERMINAL 1][TMUX 0]:** `roslaunch turtlebot3_bringup turtlebot3_robot.launch`  
Once you see the following text:  
        Calibration End  
then the robot core packages are up and running
1. Next, go to **[TERMINAL 2]** and create a new tab using the `CTRL+SHIFT+T` keyboard shortcut (we'll call this new tab **[TERMINAL 2][TAB 2]**)  
1. Finally, we need to launch some more *ROS packages* **on the laptop**, so in the new tab you have just created, enter the following:  
**[TERMINAL 2][TAB 2]:** `roslaunch turtlebot3_bringup turtlebot3_remote.launch`  
The following message should appear to indicate that the remote packages on the *laptop* have been launched sucessfully:  
        process[robot_state_publisher-1]: started with pid [xxxx]  

**The connection between our robot and the laptop is now established, *ROS master* is up and running on the laptop, and a number of *ROS Nodes* have been launched on both the robot and laptop, so the TurtleBot3 Robot is now ready to go!**

## Exercise 1: Making the TurtleBot move

1. Go to **[TERMINAL 2][TAB 2]** and enter `CTRL+SHIFT+T` to open *another* new tab (which we'll call **[TERMINAL 2][TAB 3]** - are you loosing track yet???)  
1. Enter the following:  
**[TERMINAL 2][TAB 3]:** `roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch`  
and you should see the following:
        Control Your TurtleBot3!
        ---------------------------
        Moving around:
                w
           a    s    d
                x

        w/x : increase/decrease linear velocity (Burger : ~ 0.22, Waffle and Waffle Pi : ~ 0.26)
        a/d : increase/decrease angular velocity (Burger : ~ 2.84, Waffle and Waffle Pi : ~ 1.82)

        space key, s : force stop

        CTRL-C to quit  
1. Follow the instructions in the terminal to drive the robot around!      
1. Once you've had enough, **stop the process** by entering `CTRL+C` in **[TERMINAL 2][TAB 3]** (but leave the tab open)  
1. Then go to **[TERMINAL 2][TAB 2]** and use `CTRL+C` to stop the process running there too

## Exercise 2: Building a Map

1. Make sure that the processes running in **[TERMINAL 1][TMUX 0]**, and **[TERMINAL 2][TAB 1]** are still active  
1. Go back to **[TERMINAL 2][TAB 2]** and enter the following:  
**[TERMINAL 2][TAB 2]:** `roslaunch turtlebot3_slam turtlebot3_slam.launch slam_methods:=gmapping`  
This will launch a program called *RViz*, and you should see something similar to the screenshot below:<img src="figures/slam1.png">  
1. Drive the robot around using the bluetooth remote to build up a complete map of the area:<img src="figures/slam23.png">  
**Note:** *It's best to do this slowly and perform multiple circuits of the whole area to build up a more accurate map.*  
1. Once you've made a nice map, save it as follows:  
**[TERMINAL 2][TAB 3]:** `rosrun map_server map_saver -f ~/waffle_demo/data/map`  
1. Go back to **[TERMINAL 2][TAB 2]** and stop the process using `CTRL+C`.  The RViz window should close down.

## Exercise 3: Autonomous Navigation
You can use the map that you have just created to actually perform autonomous navigation of the arena!  
1. Enter the following in **[TERMINAL 2][TAB 2]** (it's all one long command so make sure you copy and paste it all):  
**[TERMINAL 2][TAB 2]:** `roslaunch turtlebot3_navigation turtlebot3_navigation.launch map_file:=$HOME/waffle_demo/data/map.yaml`  
RViz should then open up again, this time with a top-down view of our robot within the map that you generated earlier (in black).  
**Note:** *To begin with, the robot might not actually be located correctly within the map*
1.  First we need to get the pose of the robot correct:  
    1. Press the "2D Pose Estimate" button at the top of the RViz screen  
    1. The map file should be visible (in black) in the background underneath all the live data from the robot.  Move the cursor to the point on the background map where you think the robot is actually located  
    1. Press and hold the left mouse button and a large green arrow will appear  
    1. Whilst still holding down the left mouse button, rotate the green arrow to match the direction of the robot within the map  
    1. Let go of the left mouse button to set the initial robot pose.  If you've done it correctly, the real-time LiDAR data should now match with the background map, if not then have another go<img src="figures/nav12.png">  
1. We then need to move the robot around a bit to gather some more information about the environment so that the robot can determine it's actual position more accurately.  
**Note:** *The cloud of small green arrows around the robot shows where the robot thinks it is.  We need to reduce the scatter in this by moving the robot around a bit.*  
Move the robot around using the bluetooth controller until the particle cloud begins to converge underneath the robot<img src="figures/nav3.png">  
1. Now, click the "2D Nav Goal" button:  
    1. Move the cursor to the location that you want the robot to move to  
    1. Click and hold the left mouse button and a large green arrow will appear again, the base of the arrow is the *position goal*  
    1. Whilst still holding the left mouse button, rotate the green arrow around to set the desired *orientation goal* for the robot  
    1. Release the mouse button to set the goal, and watch the robot try to navigate to the destination on it's own!

## Shutting Down
1. First, close down all the active processes that are still running in all the terminals using `CTRL+C`  
1. In **[TERMINAL 1][TMUX 0]** enter the `CTRL+B`,`D` keyboard combination to disconnect from the tmux session  
1. Then, shutdown the robot by entering the following command in **[TERMINAL 1]**:  
**[TERMINAL 1]:** `sudo shutdown -P now`  
1. Enter the password if/when asked, then wait for the message:
        Connection to dia-waffleX closed  
1. Finally, turn off the robot via the power switch on the front.

Tom Howard  
The University of Sheffield   
Last Updated: 13/02/2020